In [1]:
## UV decomposition using vectorized operations and Alternating least Squares method

import numpy as np
import pandas as pd

In [8]:
M = np.array([[1,2,3,4],[np.nan,2,np.nan,4],[5,np.nan,np.nan,5],[np.nan,2,3,4]])
M
#np.eye(10,10)

array([[  1.,   2.,   3.,   4.],
       [ nan,   2.,  nan,   4.],
       [  5.,  nan,  nan,   5.],
       [ nan,   2.,   3.,   4.]])

In [5]:
#Get the error
def get_error(M,U,V):
    return np.sqrt(np.nanmean(np.square(M - np.dot(U,V))))

In [ ]:
def compute_U_row(i,M,V,d,lambda1):
    num_of_users, num_of_items = M.shape
    print "num_of_users = {} and num_of_items = {}".format(num_of_users, num_of_items)
    
    np.eye(num_of_items,num_of_items) * lambda1 + V[ ]

## Alternating Least Squares Algorithm

_Objective_: Express the matrix M as a product of U and V. M is a mXn matrix, U is a mXd matrix and V is a dXn matrix, where d is the number of latent factors.


Mathematically, let
$$M=\left[ \begin{array}{cccccc} r_{11} & r_{12} & . & . & r_{1n} \\ r_{21} & r_{22} & . & . & r_{2n} \\ r_{31} & r_{32} & . & . & r_{3n} \\ . & . & . & . & . \\  . & . & . & . & . \\ r_{m1} & r_{m2} & . & . & r_{mn} \end{array} \right],U=\left[ \begin{array}{ccc} u_{11} & . & u_{1d} \\ u_{21} & . & u_{2d} \\  u_{31} & . & u_{3d} \\ . & . & . \\ .& . & . \\ u_{m1} & . & u_{md} \end{array} \right], V=\left[ \begin{array}{ccccc} v_{11} & v_{12} & . & . & v_{1n} \\ v_{21} & v_{22} & . & . & v_{2n} \\ . & . & . & . & . \\v_{d1} & v_{d2} & . & . & v_{dn}\end{array} \right]
$$

Then M can be expressed as the matrix product of U and V, as shown below: 
$$\left[ \begin{array}{cccccc} r_{11} & r_{12} & . & . & r_{1n} \\ r_{21} & r_{22} & . & . & r_{2n} \\ r_{31} & r_{32} & . & . & r_{3n} \\ . & . & . & . & . \\  . & . & . & . & . \\ r_{m1} & r_{m2} & . & . & r_{mn} \end{array} \right] = 
\left[ \begin{array}{ccc} u_{11} & . & u_{1d} \\ u_{21} & . & u_{2d} \\  u_{31} & . & u_{3d} \\ . & . & . \\ .& . & . \\ u_{m1} & . & u_{md} \end{array} \right] . \left[ \begin{array}{ccccc} v_{11} & v_{12} & . & . & v_{1n} \\ v_{21} & v_{22} & . & . & v_{2n} \\ . & . & . & . & . \\v_{d1} & v_{d2} & . & . & v_{dn}\end{array} \right] $$


In recommender systems, the matrix M is a sparse matrix, as shown below:

$$M=\left[ \begin{array}{cccccc}  & r_{12} & . & . & r_{1n} \\ r_{21} &   & . & . & r_{2n} \\ r_{31} &   & . & . &   \\ . & . & . & . & . \\  . & . & . & . & . \\ r_{m1} & r_{m2} & . & . &   \end{array} \right]$$

Our main objective is to estimate the U and V matrices considering only the available data in M. One method to estimate the U and V matrices is _Alternating Least Squares (ALS)_ method. The algorithm for ALS is given below:

1. Initialize U and V matrices to random real numbers
2. Let i represent row number in U and j represent the column number in V
3. To estimate the row $U_i$, perform the following:

   3a. Get the list of all columns in $M_i$ row, where we have available values. Call these locations as C

   3b. Let $V_C = V[:,C]$, where $V[:,C]$ is the list of all columns in V, corresponding to the column numbers present in C
   
4. Estimate new value of $U_i$ as:
$$U^{new}_{i} = {(\lambda_1.I + V_C . V^T_C)}^{-1} (V_C . M^T_i)$$

5. Update $U_i = {(U^{new}_{i})}^T$

6. To estimate the column $V_j$, perform the following:
   
   6a. Get the list of all rows in $M_j$ column, where we have available values. Call these locations as R
   
   6b. Let $U_R = U[R,:]$, where $U[R,:]$ is the list of all rows in U, corresponding to the row numbers present in R

7. Estimate new value of $V_j$ as:
$$V^{new}_{j} = {(\lambda_2.I + V_R . V^T_R)}^{-1} (U_R . M^T_j)$$

8. Update $V_j = {(V^{new}_j)}^T$

9. Get the U.V 

10. Subtract the available values of R from the corresponding values of U.V. Square the resulting values, sum them and take the square root to obtain RMSE (Root Mean Squared Error)

11. Repeat steps 3 to 10, until desired number of iterations or until RMSE is stablized

Note: The $\lambda_1$ and $\lambda_2$ are regularization parameters. We have to tune these parameters to optimal values. 